# PhaseObjC - Stoichiometric Phase Example 1
## Illustrate Objective-C type library interaction and plotting
Required Python code to load the phase library. 

In [ ]:
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))

### Create a Python reference to the `ForsteriteHollandAndPowell` stoichiometric phase class, and instantiate an instance of that class. 

In [ ]:
#Forsterite = ObjCClass('ForsteriteHollandAndPowell')
#obj = Forsterite.alloc().init()
Quartz = ObjCClass('QuartzHollandAndPowell')
obj = Quartz.alloc().init()

### Obtain properties of the phase inherited from the `PhaseBase` class.  
Molecular weight in grams/mole

In [ ]:
print (obj.phaseName)
print (obj.phaseFormula)
print (obj.mw)

Number of moles of each element calculated from the phase formula.  
@property (readonly) DoubleVector *formulaAsElementArray;  

Note that the class method, `elementNameFromAtomicNumber`, from the `PhaseBase` base class is used to construct a formula directly from the moles vector.

In [ ]:
PhaseBase = ObjCClass('PhaseBase')
moles_pointer = obj.formulaAsElementArray
ne = obj.formulaAsElementArray.size
formula = ''
for i in range(0, ne):
    value = obj.formulaAsElementArray.valueAtIndex_(i)
    if value != 0.0:
        name = PhaseBase.elementNameFromAtomicNumber_(i)
        formula = formula + name + '(' + str(value) + ')'
print (formula)

## All phases that conform to the Stoichiometric Phase Protocol ...
### ...implement the following functions:
```
(double)getGibbsFreeEnergyFromT:(double)t andP:(double)p;
(double)getEnthalpyFromT:(double)t andP:(double)p;
(double)getEntropyFromT:(double)t andP:(double)p;
(double)getHeatCapacityFromT:(double)t andP:(double)p;
(double)getDcpDtFromT:(double)t andP:(double)p;
(double)getVolumeFromT:(double)t andP:(double)p;
(double)getDvDtFromT:(double)t andP:(double)p;
(double)getDvDpFromT:(double)t andP:(double)p;
(double)getD2vDt2FromT:(double)t andP:(double)p;
(double)getD2vDtDpFromT:(double)t andP:(double)p;
(double)getD2vDp2FromT:(double)t andP:(double)p;
```
where *t* (temperature) is in K, and *p* (pressure) is in bars.
### In Python, these calls are written:

In [ ]:
t = 1000.0
p = 1000.0
print ("{0:>10s}{1:15.2f}{2:<20s}".format("G", obj.getGibbsFreeEnergyFromT_andP_(t, p), ' J/mol'))
print ("{0:>10s}{1:15.2f}{2:<20s}".format("H", obj.getEnthalpyFromT_andP_(t, p), ' J/mol'))
print ("{0:>10s}{1:15.2f}{2:<20s}".format("S", obj.getEntropyFromT_andP_(t, p), ' J/K-mol'))
print ("{0:>10s}{1:15.3f}{2:<20s}".format("Cp", obj.getHeatCapacityFromT_andP_(t, p), ' J/K-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("dCp/dT", obj.getDcpDtFromT_andP_(t, p), ' J/-K^2-mol'))
print ("{0:>10s}{1:15.3f}{2:<20s}".format("V", obj.getVolumeFromT_andP_(t, p), ' J/bar-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("dV/dT", obj.getDvDtFromT_andP_(t, p), ' J/bar-K-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("dv/dP", obj.getDvDpFromT_andP_(t, p), ' J/bar^2-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("d2V/dT2", obj.getD2vDt2FromT_andP_(t, p), ' J/bar-K^2-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("d2V/dTdP", obj.getD2vDtDpFromT_andP_(t, p), ' J/bar^2-K-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("d2V/dP2", obj.getD2vDp2FromT_andP_(t, p), ' J/bar^3-mol'))

In [ ]:
from scipy.misc import derivative
t = 1000.0
p = 1000.0

In [ ]:
def test_h(t,p):
    h_est = obj.getGibbsFreeEnergyFromT_andP_(t,p) + t*obj.getEntropyFromT_andP_(t, p)
    h_act = obj.getEnthalpyFromT_andP_(t, p)
    h_err = (h_est-h_act)*100.0/h_act
    print ("H       {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(h_err, h_est, h_act))
def g(x, doT=True):
    if doT:
        return obj.getGibbsFreeEnergyFromT_andP_(x, p)
    else:
        return obj.getGibbsFreeEnergyFromT_andP_(t, x)
def test_g_dt(t,p):
    s_est = -derivative(g, t, args=(True,))
    s_act = obj.getEntropyFromT_andP_(t, p)
    s_err = (s_est-s_act)*100.0/s_act
    print ("S       {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(s_err, s_est, s_act))
def test_g_dp(t,p):
    v_est = derivative(g, p, args=(False,))
    v_act = obj.getVolumeFromT_andP_(t, p)
    v_err = (v_est-v_act)*100.0/v_act
    print ("V       {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(v_err, v_est, v_act))
def s(x, doT=True):
    if doT:
        return obj.getEntropyFromT_andP_(x, p)
    else:
        return obj.getEntropyFromT_andP_(t, x)
def test_s_dt(t,p):
    cp_est = t*derivative(s, t, args=(True,))
    cp_act = obj.getHeatCapacityFromT_andP_(t, p)
    cp_err = (cp_est-cp_act)*100.0/cp_act
    print ("Cp      {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(cp_err, cp_est, cp_act))
def cp(x, doT=True):
    if doT:
        return obj.getHeatCapacityFromT_andP_(x, p)
    else:
        return obj.getHeatCapacityFromT_andP_(t, x)
def test_cp_dt(t,p):
    dcpdt_est = derivative(cp, t, args=(True,))
    dcpdt_act = obj.getDcpDtFromT_andP_(t, p)
    dcpdt_err = (dcpdt_est-dcpdt_act)*100.0/dcpdt_act
    print ("dCpDt   {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(dcpdt_err, dcpdt_est, dcpdt_act))
def v(x, doT=True):
    if doT:
        return obj.getVolumeFromT_andP_(x, p)
    else:
        return obj.getVolumeFromT_andP_(t, x)
def test_v_dt(t,p):
    dvdt_est = derivative(v, t, args=(True,))
    dvdt_act = obj.getDvDtFromT_andP_(t, p)
    dvdt_err = (dvdt_est-dvdt_act)*100.0/dvdt_act
    print ("dVdT    {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(dvdt_err, dvdt_est, dvdt_act))
def test_v_dp(t,p):
    dvdp_est = derivative(v, p, args=(False,))
    dvdp_act = obj.getDvDpFromT_andP_(t, p)
    dvdp_err = (dvdp_est-dvdp_act)*100.0/dvdp_act
    print ("dVdP    {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(dvdp_err, dvdp_est, dvdp_act))
def dvdt(x, doT=True):
    if doT:
        return obj.getDvDtFromT_andP_(x, p)
    else:
        return obj.getDvDtFromT_andP_(t, x)
def dvdp(x, doT=True):
    if doT:
        return obj.getDvDpFromT_andP_(x, p)
    else:
        return obj.getDvDpFromT_andP_(t, x)
def test_dvdt_dt(t,p):
    d2vdt2_est = derivative(dvdt, t, args=(True,))
    d2vdt2_act = obj.getD2vDt2FromT_andP_(t, p)
    d2vdt2_err = (d2vdt2_est-d2vdt2_act)*100.0/d2vdt2_act
    print ("d2VdT2  {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdt2_err, d2vdt2_est, d2vdt2_act))
def test_dvdt_dp(t,p):
    d2vdtdp_est = derivative(dvdt, p, args=(False,))
    d2vdtdp_act = obj.getD2vDtDpFromT_andP_(t, p)
    d2vdtdp_err = (d2vdtdp_est-d2vdtdp_act)*100.0/d2vdtdp_act
    print ("d2VdTdP {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdtdp_err, d2vdtdp_est, d2vdtdp_act))

def test_dvdp_dt(t,p):
    d2vdtdp_est = derivative(dvdp, t, args=(True,))
    d2vdtdp_act = obj.getD2vDtDpFromT_andP_(t, p)
    d2vdtdp_err = (d2vdtdp_est-d2vdtdp_act)*100.0/d2vdtdp_act
    print ("d2VdTDp {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdtdp_err, d2vdtdp_est, d2vdtdp_act))
def test_dvdp_dp(t,p):
    d2vdp2_est = derivative(dvdp, p, args=(False,))
    d2vdp2_act = obj.getD2vDp2FromT_andP_(t, p)
    d2vdp2_err = (d2vdp2_est-d2vdp2_act)*100.0/d2vdp2_act
    print ("d2VdP2  {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdp2_err, d2vdp2_est, d2vdp2_act))

In [ ]:
test_h(t,p)
test_g_dt(t,p)
test_s_dt(t,p)
test_cp_dt(t,p)
test_g_dp(t,p)
test_v_dt(t,p)
test_v_dp(t,p)
test_dvdt_dt(t,p)
test_dvdt_dp(t,p)
test_dvdp_dt(t,p)
test_dvdp_dp(t,p)

In [ ]:
derivative?